<a href="https://colab.research.google.com/github/RachitBansal/Transformer-NMT/blob/master/English2Hindi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!wget http://www.cfilt.iitb.ac.in/~moses/iitb_en_hi_parallel/iitb_corpus_download/parallel.tgz

--2020-04-07 17:20:50--  http://www.cfilt.iitb.ac.in/~moses/iitb_en_hi_parallel/iitb_corpus_download/parallel.tgz
Resolving www.cfilt.iitb.ac.in (www.cfilt.iitb.ac.in)... 103.21.127.130
Connecting to www.cfilt.iitb.ac.in (www.cfilt.iitb.ac.in)|103.21.127.130|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 96098008 (92M) [application/x-gzip]
Saving to: ‘parallel.tgz’

parallel.tgz        100%[===================>]  91.65M  8.10MB/s    in 25s     

2020-04-07 17:21:15 (3.66 MB/s) - ‘parallel.tgz’ saved [96098008/96098008]



In [13]:
!tar -xzvf parallel.tgz

parallel/
parallel/IITB.en-hi.hi
parallel/IITB.en-hi.en


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# %cp parallel/IITB.en-hi.hi drive/My\ Drive/IITB.en-hi.hi

In [0]:
# %cp parallel/IITB.en-hi.en drive/My\ Drive/IITB.en-hi.en

In [0]:
# path1 = 'drive/My Drive/IITB.en-hi.hi'
# path2 = 'drive/My Drive/IITB.en-hi.en'

In [0]:
# eng_sents = []
# hi_sents =  []
# i = 0
# with open('parallel/IITB.en-hi.hi', 'rt') as f:
#   while(i!=10**6):
#     eng_sents.append('\sep '+f.readline())
#     i+=1

# with open('parallel/IITB.en-hi.en', 'rt') as f:
#   while(i!=2*10**6):
#     hi_sents.append('\sep '+f.readline())
#     i+=1

In [55]:
# print(len(eng_sents), len(hi_sents))
# print(eng_sents[-1], hi_sents[-1])

1000000 1000000
\sep गोदाम में रखे दृष्टिबंधक माल की संख्या बदलते रहने के कारण ऋणकर्ता अनिधिक कर्ज पर निर्भर होता है। 
 \sep Due to changing quantity of hypothicated items in godown, borrower has to depend on unfunded debt.



In [0]:
import torch
from torch import nn
from torch import optim

In [37]:
!pip install tokenizers

In [0]:
from tokenizers import ByteLevelBPETokenizer

In [0]:
tokeniser_hindi = ByteLevelBPETokenizer()
tokeniser_hindi.train(['parallel/IITB.en-hi.hi'], special_tokens= ['\sep', '\pad'])

In [0]:
tokeniser_eng = ByteLevelBPETokenizer()
tokeniser_eng.train(['parallel/IITB.en-hi.en'], special_tokens= ['\sep', '\pad'])

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
vocab_size_en = tokeniser_eng.get_vocab_size()
vocab_size_hi = tokeniser_eng.get_vocab_size()

In [59]:
print(vocab_size_en, vocab_size_hi)

30000 30000


In [0]:
class model(nn.Module):
  def __init__(self, s_lang = 'en', t_lang = 'hi'):
    super(model, self).__init__()
    if(s_lang == 'en'):
      vocab_size_src = vocab_size_en
      vocab_size_tgt = vocab_size_hi
    elif(s_lang == 'hi'):
      vocab_size_src = vocab_size_hi
      vocab_size_tgt = vocab_size_en
    self.emb_src = nn.Embedding(vocab_size_src, 512)
    self.emb_tgt = nn.Embedding(vocab_size_tgt, 512)
    enc_layer = nn.TransformerEncoderLayer(512, 8, 2048, 0.1, 'relu')
    self.transf_enc = nn.TransformerEncoder(enc_layer, 6)
    dec_layer = nn.TransformerDecoderLayer(512, 8, 2048, 0.1, 'relu')
    self.transf_dec = nn.TransformerDecoder(dec_layer, 6)
    self.softmax = nn.Softmax()
    self.linear = nn.Linear(512, vocab_size_tgt)

  def get_memory_mask(self, max_len_src, max_len_tgt):
    mask = torch.zeros(max_len_src, max_len_tgt)
    for i in range(min(max_len_src-1, max_len_tgt-1)):
      mask[i][i:] = -float('Inf')
    return mask

  def get_tgt_mask(self, max_len):
    mask = torch.zeros(max_len, max_len)
    for i in range(max_len-1):
      mask[i][i+1:] = -float('Inf')
    return mask

  def forward(self, data_, i):
    batch_x, batch_y = data_.getitem(i)
    # print(batch_x.shape, batch_y.shape)
    # batch_x = batch_x.transpose(0, 1)
    # batch_y = batch_y.transpose(0, 1)
    # mmask = self.get_memory_mask(batch_y.shape[1], batch_x.shape[1])
    tmask = self.get_tgt_mask(batch_y.shape[1])
    x = self.emb_src(batch_x)
    y = self.emb_tgt(batch_y)
    x = x.transpose(0,1)
    y = y.transpose(0,1)
    x = self.transf_enc(x)
    # print(y.shape, x.shape, mmask.shape, tmask.shape)
    x = self.transf_dec(tgt = y, memory = x, tgt_mask = tmask)
    x = self.softmax(self.linear(x))
    
    return x

In [0]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [0]:
from torch.nn.utils.rnn import pad_sequence as padd

In [0]:
class data(Dataset):
  def __init__(self, data, batch_size = 32, max_len = 80, src_lang = 'en', tar_lang = 'hi'):
    self.max_len = max_len
    self.bsz = batch_size
    self.data = data.values
    self.src_lang = src_lang
    self.tar_lang = tar_lang

  def batchify(self, i):
    batch_x  = []
    batch_y = []
    for j in range(self.bsz):
      batch_x.append(self.data[i*self.bsz + j, 0])
      batch_y.append(self.data[i*self.bsz + j, 1])

    return batch_x, batch_y
    
  def getitem(self, i):
    x, y = self.batchify(i)
    return padd(x, True, tokeniser_eng.encode('\pad').ids[0]), padd(y, True, tokeniser_eng.encode('\pad').ids[0])

In [0]:
import pandas as pd

In [0]:
def load():
    i = 0
    eng_sents = []
    hi_sents =  []
    i = 0
    with open('./parallel/IITB.en-hi.en', 'rt') as f:
      while(i!=10**6):
        sent = tokeniser_eng.encode('\sep '+f.readline()).ids
        eng_sents.append(torch.tensor(sent))
        i+=1

    with open('./parallel/IITB.en-hi.hi', 'rt') as f:
      while(i!=2*10**6):
        hi_sents.append(torch.tensor(tokeniser_hindi.encode('\sep '+f.readline()).ids))
        i+=1

    zipped_list = list(zip(eng_sents, hi_sents))
    df_prllel_en_hi = pd.DataFrame(zipped_list, columns = ['en', 'hi'], dtype=object)
    zipped_list = list(zip(hi_sents, eng_sents))
    df_prllel_hi_en = pd.DataFrame(zipped_list, columns = ['hi', 'en'], dtype=object)
    return df_prllel_en_hi, df_prllel_hi_en

d_e_h, d_h_e = load()

In [0]:
# def batchify(data, bsz = 32, src_lang = 'en', tar_lang = 'hi'):
#   return {'x':data.loc[i*bsz:(i+1)*bsz-1, src_lang], 'y':self.data.loc[i*bsz:(i+1)*bsz-1, tar_lang]}

# def batch_tokens(data, i, s_sz = 32, key = 'x'):
#   return padd([data[i][key] for i in range(b_sz)], batch_first=True, padding_value=0)

# def return_batch(data, b_sz, i):
#   padd()

In [0]:
s_lang = 'en'
t_lang = 'hi'
batch_size = 32
batchs = d_e_h.shape[0]//batch_size
data_ = data(d_e_h, batch_size, src_lang=s_lang, tar_lang=t_lang)

In [0]:
def greedy(x, i, batch_first = False):
  if(batch_first != True):
    x = x.transpose(0, 1)
  ans = torch.zeros(x.shape[0], x.shape[1])
  for i in range(x.shape[0]):
    for j in range(x[i].shape[0]):
      ans[i][j] = torch.max(x[i][j], 0)[1]
  print(ans.shape)
  return ans

In [121]:
batch_x, batch_y = data_.getitem(i)
a = []
for i in range(batch_x.shape[1]):
  a.append(batch_x[21][i])

print(tokeniser_eng.decode(a))

 Browse the various methods of the current accessible



In [0]:
model_ = model(s_lang, t_lang)
optimizer = optim.Adam(model_.parameters(), lr=0.1)
criterion = nn.CrossEntropyLoss()
for i in range(batchs):
  batch_x, batch_y = data_.getitem(i)
  x = batch_x
  y = batch_y
  # print(x)
  x = x.transpose(0,1)
  y = y.transpose(0,1)
  # print(y.shape)
  optimizer.zero_grad()
  out = model_.forward(data_, i)
  # print(out)
  loss = criterion(out.reshape(-1, vocab_size_hi), y.reshape(-1))
  loss.backward()
  optimizer.step()

  if(i%20):
    print(loss)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor(10.3067, grad_fn=<NllLossBackward>)
tensor(10.3090, grad_fn=<NllLossBackward>)
tensor(10.3078, grad_fn=<NllLossBackward>)
tensor(10.3091, grad_fn=<NllLossBackward>)
tensor(10.3086, grad_fn=<NllLossBackward>)
tensor(10.3099, grad_fn=<NllLossBackward>)
tensor(10.3122, grad_fn=<NllLossBackward>)
tensor(10.3082, grad_fn=<NllLossBackward>)
tensor(10.3084, grad_fn=<NllLossBackward>)
tensor(10.3095, grad_fn=<NllLossBackward>)
tensor(10.3089, grad_fn=<NllLossBackward>)
tensor(10.3101, grad_fn=<NllLossBackward>)
tensor(10.3088, grad_fn=<NllLossBackward>)
tensor(10.3139, grad_fn=<NllLossBackward>)
tensor(10.3113, grad_fn=<NllLossBackward>)
tensor(10.3121, grad_fn=<NllLossBackward>)
tensor(10.3097, grad_fn=<NllLossBackward>)
tensor(10.3103, grad_fn=<NllLossBackward>)
tensor(10.3131, grad_fn=<NllLossBackward>)
tensor(10.3095, grad_fn=<NllLossBackward>)
